In [ ]:
from math import tan, pi
from scipy.optimize import root
import numpy as np

from mfp2mach import mach2mfp, mfp2mach
import compressor_losses as losses

In [ ]:
class Compressor():
    def __init__(self):
        geom = {
        #Measured parameters
        'b': 5e-3,
        'D2': 64e-3,
        'D1t': 42.8e-3,
        'D1h': 17e-3,
        'beta2': 10*pi/180,
        'beta1': 40*pi/180,
        'alfa3': 69*pi/180,
        'Z': 10,
        }

        # Add derived geometry to geom
        geom['A1'] = pi/4*(geom['D1t']**2-geom['D1h']**2)
        geom['A2'] = pi*geom['D2']*geom['b']
        geom['A_ratio'] = geom['A2']/geom['A1']
        geom['R_ratio'] = geom['D2']/((geom['D1h']**2 + geom['D1t']**2)/2)**0.5
        geom['D1t_D2'] = geom['D1t']/geom['D2']
        geom['D1h_D2'] = geom['D1h']/geom['D2']
        
        self.geom=geom
        
        self.gam=1.4
    
    def implicit_map(self, MFP1, MFP2, Mb, T0_ratio, P0_ratio, tol=1e-16):
        #Unpack constants
        gam = self.gam
        slip_factor = 0.9
        beta2 = self.geom['beta2']
        A_ratio = self.geom['A_ratio']        
    
        #Non linear model
        M1 = mfp2mach(MFP1, gam, tol)
        M2 = mfp2mach(MFP2, gam, tol)
        
        phi1 = MFP1/Mb*(1+(gam-1)/2*M1**2)**(1/(gam-1))
        phi2 = MFP2/Mb*(1+(gam-1)/2*M2**2)**(1/(gam-1))*T0_ratio**0.5
        
        psi_euler = slip_factor*(1-phi2*tan(beta2))
        loss_internal = losses.internal(slip_factor=slip_factor, phi1=phi1, phi2=phi2, psi_euler=psi_euler, **self.geom)
        loss_parasitic = losses.parasitic(slip_factor=slip_factor, phi1=phi1, phi2=phi2, psi_euler=psi_euler, **self.geom)
        psi_isen = psi_euler-sum(loss_internal.values())
        psi_actual = psi_euler+sum(loss_parasitic.values())
        
        res_T0_ratio = T0_ratio - ((gam-1)*psi_actual*Mb**2 + 1)
        res_P0_ratio = P0_ratio - ((gam-1)*psi_isen*Mb**2 + 1)**(gam/(gam-1))
        res_MFP = MFP2 - MFP1*T0_ratio**0.5/(A_ratio*P0_ratio)
        
        return res_MFP, res_T0_ratio, res_P0_ratio
    
    def general_explicit_map(self, params, initial_guesses=None):
        
        default_params = {'MFP1':0.2, 'MFP2':0.2, 'Mb':1, 'T0_ratio':1, 'P0_ratio':1}
        
        if initial_guesses is None:
            initial_guesses = {k: v for k,v in default_params.items() if k not in params.keys()}
        
        assert len(params) == 2, "exactly two parameters are required"
        assert len(initial_guesses) == 3, "exactly three initial guesses are required"
        assert params.keys() | initial_guesses.keys() == default_params.keys(), "missing value for {}".format(default_params.keys() - (params.keys() | initial_guesses.keys()))
        assert params.keys().isdisjoint(initial_guesses.keys()), "A parameter cannot be supplied as both a initial guess and a fixed parameter"
        
        def fsv(x):
            kwargs = params
            kwargs.update(dict(zip(initial_guesses.keys(), x)))
            res = self.implicit_map(**kwargs)
            
            return res
          
        sol = root(fsv, list(initial_guesses.values()))  
        params.update(dict(zip(initial_guesses.keys(), sol.x)))
        sol.params=params
        
        return sol
                
    
    def explicit_map(self, MFP1, P0_ratio, initial_guess=[0.1, 1, 0.1]):
        dependent_vars = ('Mb', 'T0_ratio', 'MFP2')
        param = dict(zip(dependent_vars, range(len(dependent_vars))))
        
        def fsv(x):
            MFP2 = x[param['MFP2']]
            Mb = x[param['Mb']]
            T0_ratio = x[param['T0_ratio']]
            
            res = self.implicit_map(MFP1, MFP2, Mb, T0_ratio, P0_ratio)
            
            return res
            
        sol = root(fsv, initial_guess,options={'eps': 1e-8})
        sol.param = param
        
        return sol

    
    def outlet_choke(self, MFP1, x0=[0.1, 1, 1]):
        dependent_vars = ('Mb', 'T0_ratio', 'P0_ratio')
        param = dict(zip(dependent_vars, range(len(dependent_vars))))
        
        def fsv(x):
            Mb = x[param['Mb']]
            T0_ratio=x[param['T0_ratio']]
            P0_ratio=x[param['P0_ratio']]
            
            MFP2 = mach2mfp(1,self.gam)
            
            res = self.implicit_map(MFP1, MFP2, Mb, T0_ratio, P0_ratio)
            
            return res
            
        sol = root(fsv, x0, options={'eps': 1e-8})
        sol.param = param    
        
        return sol
    
    def critical_choke(self):
        dependent_vars = ('Mb', 'T0_ratio', 'P0_ratio')
        param = dict(zip(dependent_vars, range(len(dependent_vars))))
        
        def fsv(x):
            Mb = x[param['Mb']]
            T0_ratio=x[param['T0_ratio']]
            P0_ratio=x[param['P0_ratio']]
            
            MFP1 = mach2mfp(1,self.gam)
            MFP2 = mach2mfp(1,self.gam)
            
            res = self.implicit_map(MFP1, MFP2, Mb, T0_ratio, P0_ratio)
            
            return res
            
        sol = root(fsv, [0.1, 1, 1],options={'eps': 1e-8})
        sol.param = param      
        
        return sol 
    

In [ ]:
c = Compressor()
c.implicit_map(MFP1=-0.1, MFP2=0.1, Mb=0.1, P0_ratio=1., T0_ratio=1)

In [ ]:
c.general_explicit_map(params={'MFP2': 0.1, 'MFP1': 0.1})#, initial_guesses={'MFP2':0.2, 'Mb':1, 'T0_ratio':1})

In [ ]:
c.outlet_choke(0.5)

In [ ]:
sol_cr=c.critical_choke()
sol_cr

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
import numpy as np

In [ ]:
plt.plot(mach2mfp(1,1.4), sol_cr.x[sol_cr.param['P0_ratio']], '*')
samples=100
P0_max = 4
MFP1_grid = np.empty((samples, samples))
P0_grid = np.empty_like(MFP1_grid)
x0 =sol_cr.x
for i, MFP1 in enumerate(np.linspace(mach2mfp(1,1.4),0, samples)):
    sol = c.outlet_choke(MFP1, x0)
    x0=sol.x
    P0_choke = sol.x[sol.param['P0_ratio']]
    _=plt.plot(MFP1, P0_choke, '.')
    MFP1_grid[i, :] = MFP1
    P0_grid[i, :] = np.linspace(P0_max, P0_choke if P0_choke>1 else 1, samples)
    
_=plt.plot(MFP1_grid, P0_grid, '+')
    

In [ ]:
success=np.empty_like(MFP1_grid)
Mb_grid=np.empty_like(MFP1_grid)
nfev=np.empty_like(MFP1_grid)
eff_grid=np.empty_like(MFP1_grid)
x0 = [0.4, 1.1, 0.4]
for i in range(samples):
    for j in range(samples):
        sol = c.explicit_map(MFP1_grid[i,j], P0_grid[i,j], x0)
#         x0=sol.x if sol.success else [0.4, 1.1, 0.4]
        success [i,j] = sol.success
        nfev[i,j] = sol.nfev
        Mb = sol.x[sol.param['Mb']]
        Mb_grid[i,j] = Mb if sol.success else np.nan
        eff = (c.gam-1)/c.gam*np.log(P0_grid[i,j])/np.log(sol.x[sol.param['T0_ratio']])
        eff_grid[i,j] = eff if eff<=1 else np.nan
        
plt.imshow(success)


In [ ]:
plt.imshow(nfev)

In [ ]:
plt.imshow(Mb_grid)

In [ ]:
plt.imshow(eff_grid)

In [ ]:
plt.figure(figsize=(6,8),dpi=150)
CS2 = plt.contour(MFP1_grid, P0_grid, Mb_grid, levels=np.arange(0,2,0.1), colors='k', linewidths=1.5)
plt.clabel(CS2, CS2.levels, fmt='%.1f')
# plt.plot(MFP1_grid, P0_grid, '+')
CS2 = plt.contour(MFP1_grid, P0_grid, eff_grid, colors='k', linewidths=0.5, levels=[0.5,0.8,0.9,0.95])
plt.clabel(CS2, CS2.levels, fmt='%.2f')
plt.xlim((0,0.6))
plt.ylim((1,4))
plt.xlabel("MFP")
plt.ylabel(r"$\frac{P_{03}}{P_{02}}$")
plt.legend(("Mb", "eta_p"))
plt.savefig('map.pdf', dpi=300)


In [ ]:
(np.nanmax(eff_grid),np.nanmin(eff_grid))

In [ ]:
# Define map corners
# Define grid
# Calculate coarse grid
# Interpolate to fine grid
# Calculate fine grid

In [ ]:
a = {'a':0, 'b':1}
b = {'c':2, 'a':1}

In [ ]:
a.keys()-b.keys()

In [ ]:
help(a.keys())

In [ ]:
a[b.keys()]